In [1]:
%load_ext autoreload
%autoreload 2

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings("ignore")

from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import SMOTENC
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
from sklearn.svm import LinearSVC

from catboost import CatBoostClassifier
import lightgbm as lgb
from xgboost import XGBClassifier

import data_preprocessing as data_prep
from renameFeatures import rename_features, get_data_summary, get_data_summary2
from data_vizualization import metrics_plotting, metrics_plotting_with_events
import modelling

In [4]:
plt.rcParams["figure.figsize"] = (15, 10)

In [3]:
results = pd.read_csv("./test_answers.csv", parse_dates=["time"])
results = results.drop("Unnamed: 0", axis=1)
results = results.reset_index(drop=True)

In [5]:
results.head()

,active_power,electricity_gage,event_id,frequency,intake_pressure,line_pressure,liquid_rate,motor_load,pump_temperature,time,...,pump_temperature_rol_std,pump_temperature_rol_max,pump_temperature_rol_min,pump_temperature_spk,failure_target,Well__1,Well__4,Well__7,Well__8,predicted
0,72.010030,1765.09,0,95.11465,27.859285,20.529285,104.4,54.800000,96.0,2021-05-01 00:00:00,...,0.40406,97.0,95.0,1.00194,0,0,1,0,0,False
1,72.058550,1765.09,0,95.09646,27.855715,20.525715,104.4,54.830770,96.0,2021-05-01 00:02:00,...,0.40406,97.0,95.0,1.00194,0,0,1,0,0,False
2,72.103340,1765.09,0,95.08068,27.852142,20.522142,104.4,54.858974,96.0,2021-05-01 00:04:00,...,0.40406,97.0,95.0,1.00194,0,0,1,0,0,False
3,72.144424,1765.09,0,95.06731,27.848572,20.518572,104.4,54.884617,96.0,2021-05-01 00:06:00,...,0.40406,97.0,95.0,1.00194,0,0,1,0,0,False
4,72.181786,1765.09,0,95.05635,27.845000,20.515000,104.4,54.907692,96.0,2021-05-01 00:08:00,...,0.40406,97.0,95.0,1.00194,0,0,1,0,0,False


In [8]:
modelling.evaluate_results(results["failure_target"], results["predicted"])

Precision --- 0.31645569620253167
Recall --- 0.8240096764439069
ROC_AUC --- 0.7637466079506756


In [9]:
results.columns

Index(['active_power', 'electricity_gage', 'event_id', 'frequency',
       'intake_pressure', 'line_pressure', 'liquid_rate', 'motor_load',
       'pump_temperature', 'time', 'voltage_unbalance', 'current_unbalance',
       'voltage', 'current', 'resistance', 'current_deriv', 'current_rol_mean',
       'current_rol_std', 'current_rol_max', 'current_rol_min', 'current_spk',
       'voltage_deriv', 'voltage_rol_mean', 'voltage_rol_std',
       'voltage_rol_max', 'voltage_rol_min', 'voltage_spk',
       'active_power_deriv', 'active_power_rol_mean', 'active_power_rol_std',
       'active_power_rol_max', 'active_power_rol_min', 'active_power_spk',
       'frequency_deriv', 'frequency_rol_mean', 'frequency_rol_std',
       'frequency_rol_max', 'frequency_rol_min', 'frequency_spk',
       'electricity_gage_deriv', 'electricity_gage_rol_mean',
       'electricity_gage_rol_std', 'electricity_gage_rol_max',
       'electricity_gage_rol_min', 'electricity_gage_spk', 'motor_load_deriv',
       'm

In [20]:
def permutations(string):
    if len(string) == 1: return set(string)    # Условие выхода из рекурсии
    first = string[0]                          # Первая буква в строке
    print(string)
    rest = permutations(string[1:])            # Рекурсия, вызываю функцию со строкой [1:] от изначальной
    result = set()
    for i in range(0, len(string)):            # Для всего количества имеющейся строки
        for p in rest:
            result.add(p[0:i] + first + p[i:])
    print(result)
    return result

In [21]:
string = "abab"

In [22]:
permutations(string)

abab
bab
ab
{'ab', 'ba'}
{'abb', 'bab', 'bba'}
{'abba', 'abab', 'baab', 'bbaa', 'aabb', 'baba'}


{'aabb', 'abab', 'abba', 'baab', 'baba', 'bbaa'}

In [27]:
a = "00011010"

In [28]:
a.replace("110", "b")

'000b10'

In [25]:
a

[44, 2, 3, 4]

In [26]:
sorted?

Signature: sorted(iterable, /, *, key=None, reverse=False)
Docstring:
Return a new list containing all items from the iterable in ascending order.

A custom key function can be supplied to customize the sort order, and the
reverse flag can be set to request the result in descending order.
Type:      builtin_function_or_method
